In [1]:
from os import listdir
import os
import pickle

done = 0
data = {}
for f in listdir('data'):
    data[f[:-2]] = pickle.load(open('data/' + f, 'rb'), encoding='latin1')

genes = [x for x in data]
training = genes[:5]
testing = genes[5:]

In [2]:
import random

val_p = 1
val_n = 0

pA=[('A' * 20, val_p)],
pB=[('T' * 20, val_p)],
pC=[('C' * 20, val_p)],
pD=[('G' * 20, val_p)],
nA=[('A' * 20, val_n)],
nB=[('T' * 20, val_n)],
nC=[('C' * 20, val_n)],
nD=[('G' * 20, val_n)],

toy_env = dict(
    tB = (5 * pC + 5 * pB + 10 * nD) * 2,
    tC = (5 * pA + 5 * pD + 5 * nC + 5 * nB) * 2,
    tD = (5 * nD + 5 * nB + 10 * pA) * 2,
    vB = (5 * pA + 5 * pD + 5 * nC + 5 * nB) * 2,
    tE = (5 * pC + 5 * pB + 10 * nD) * 2,
    vA = (5 * pA + 5 * pD + 5 * nC + 5 * nB) * 2,
    vC = (5 * pC + 5 * pB + 10 * nD) * 2,
    tA = (5 * nC + 5 * nB + 10 * pA) * 2,
)
data = {}
for x in toy_env:
    data[x] = [i[0] for i in toy_env[x]]
    
genes = [x for x in data]
random.shuffle(genes)
training = genes[:5]
testing = genes[5:]

In [3]:
from seq_net import *
sess = tf.Session()

In [4]:
net = SeqNet(sess)

In [ ]:
results = []

In [ ]:
for i in trange(100):
    for i in training:
        net.train([(dna_vec(a), b) for a, b in data[i]], 50)
    for i in testing:
        results.append(net.evaluate([(dna_vec(a), b) for a, b in data[i]], 10))


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
smooth = 5
graph = np.convolve(results, [1 / smooth] * smooth)[smooth:-smooth]
plt.plot(graph)

In [5]:
print(set(data[testing[0]]))

{('TTTTTTTTTTTTTTTTTTTT', 1), ('GGGGGGGGGGGGGGGGGGGG', 0), ('CCCCCCCCCCCCCCCCCCCC', 1)}


In [6]:
for i in [testing[0]]:
    s, a, r = (net.path([(dna_vec(a), b) for a, b in data[i]]))
    for z, x, y in zip(s, a, r):
        for t, p in z:
            print(vec_dna(t), p)
        print('done:', vec_dna(x), y)
    print('-' * 50)

GGGGGGGGGGGGGGGGGGGG 0
[['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['CCCCCCCCCCCCCCCCCCCC', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['TTTTTTTTTTTTTTTTTTTT', 1], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGGGGG', 0], ['GGGGGGGGGGGGGGGGG